# simple Roman Coronagraph target accessibility calculator
Calculates approximate date ranges each year when the Roman Observatory can point to a given target, based only on Sun-angle constraints.  Marks periods of time when the Galactic Bulge is observable, because the Galactic Bulge Time Domain Survey (GBTDS) will typically take priority during those times.

In [ ]:
from yp import *
from astropy.io import ascii
from astropy.table import unique as aptunique
import matplotlib.pyplot as plt
%matplotlib inline
import os
from pandas import DataFrame
from numpy import nan
import astropy.coordinates as coord


In [ ]:
# prefix for target list file name
prefix = 'example'

# make directory for output files
if not os.path.exists('./output'):
    os.mkdir('./output')

In [ ]:
# optionally...Look up coordinates for all targets in simbad. 
# If you manually make a coordinates file, you can skip this step by setting runSimbad = False. 

runSimbad = True

if runSimbad:
    from astroquery.simbad import Simbad

    dat = ascii.read(prefix+'.txt', comment='#')
    dat.meta = {}
    RA_d = []
    RA_h = []
    dec_d = []
    dec_h = []

    from time import sleep

    for ct in range(len(dat)):
        result_table = Simbad.query_object(dat['Name'][ct])
        sleep(1/6) # don't submit more than 6 quieries per second to avoid IP blacklisting
        tmp = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], frame='icrs', unit=(u.hourangle, u.deg)) 
        RA_d.append(tmp.ra.deg)
        RA_h.append(tmp.ra.to_string(unit=u.hour))
        dec_d.append(tmp.dec.deg)
        dec_h.append(tmp.dec.to_string(unit=u.deg))

    dat['RA_d'] = RA_d
    dat['dec_d']  = dec_d
    dat['RA_h'] = RA_h
    dat['dec_h']  = dec_h
    
    # The Galactic Bulge Time Domain Survey will take priority most of the time when the Bulge is observable
    gbtds = coord.SkyCoord(0, 0, unit=(u.degree, u.degree), frame='galactic').transform_to('icrs')
    dat.add_row( ['_GBTDS_', 'WFI',gbtds.ra.deg, gbtds.dec.deg, \
                gbtds.ra.to_string(unit=u.hour), gbtds.dec.to_string(unit=u.deg)] )

    ascii.write(dat,'output/coords_'+prefix+'.csv', delimiter=',', overwrite=True)


calculate when we can point at each target. ONLY accounts for solar panel-to-sun angle, not Moon/Earth/Solar System bodies keep-out zones

In [ ]:
sttab = ascii.read('output/coords_'+prefix+'.csv')
sttab['lowername'] = [str.lower(x) for x in sttab['Name']] 
sttab = aptunique(sttab, keys='Name', keep='first') 

sttab.sort('lowername')
nstars = len(sttab['Name'])
isobs = np.zeros( (365,nstars), dtype=int)

max_sunangle= 34


for ct, name in enumerate(sttab['Name']):
    tmp = (sttab[ct]['RA_d']*u.deg, sttab[ct]['dec_d']*u.deg)
    for iday in range(365):
        isobs[iday, ct] = is_observable(tmp, iday, max_dgr=max_sunangle)

fracobs = np.mean(isobs,0)


In [ ]:
with open('output/observability_'+prefix+'.csv', 'w') as f:
    f.write('%-11s %-3s\n'%('Name','frac_time_observable'))
    for ct in range(len(fracobs)):
        f.write('%-11s %3.2f\n'%(sttab[ct]['Name'], fracobs[ct]))
f.close()

In [ ]:
df = DataFrame(data=isobs, columns=sttab['Name'])
df['day number'] = np.arange(365)+1
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.to_csv('output/observability_'+prefix+'_'+str(max_sunangle)+'dgr.csv', index=False)

# start and stop days of Galactic Buldge season
tmp = df['_GBTDS_'] - np.roll(df['_GBTDS_'],1) 
gbtds_obs_days = np.argwhere(tmp.values != 0 )

In [ ]:
#fig, ax = plt.subplots( nrows=2, ncols=1, figsize = (12,15) ) 

theme = 'light'
if theme.lower() == 'light':
    cm = 'Greens'
    cl = [-0.1,1.3]
    csep = 'w'
    cf = [0,.2,0]
elif theme.lower() == 'dark':
    cm = 'hot'
    cl = [-0.1,1.1]
    plt.gca().set_facecolor('black')
    csep = 'black'
    cf = [1,1,.5]
else:
    raise Exception('color theme must be "light" or "dark"')
    

    
for sort_by_observability in [True, False]:
    fig = plt.figure(figsize=(12,7))

    if sort_by_observability == True:
        tosort = np.argsort(fracobs)[::-1]
        tmp = isobs.T[tosort,:]
        textorder = tosort
        ax = fig.add_subplot(111)
    else:
        tmp = isobs.T
        textorder = range(nstars)
        ax = fig.add_subplot(111)
        
    plt.imshow(tmp, aspect='auto', origin='upper', cmap=cm)
    plt.clim(cl)
    plt.xlim([0,430])
    
     # plot vertical lines at microlensing season boundaries
    plt.axvspan(gbtds_obs_days[0], gbtds_obs_days[1]-1, color='k', alpha=.3)
    plt.axvspan(gbtds_obs_days[2], gbtds_obs_days[3]-1, color='k', alpha=.3)
    
    # label each row with target name and % of time observable
    for ct, idx in enumerate(textorder):
        plt.text(370, ct, '%-11s%3s %3.0f%%'%(sttab[idx]['Name'], sttab[idx]['type'], fracobs[idx]*100), \
                 color=cf, verticalalignment='center', fontdict={'family':'monospace'})
        plt.axhline(ct+0.5, c=csep, linewidth=5)
    plt.axhline(-0.5, c=csep, linewidth=5)

    # make x axis tick labels more sensible
    xt = []
    monthnames = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
    xtl = []
    for ct in np.arange(12)+1:
        tmp = datetime.date(2026, ct, 1).timetuple().tm_yday
        plt.axvline(tmp-0.5, c='dimgray', linewidth=1)
        xt.append(tmp)
        xtl.append(monthnames[ct-1]+' (%i)'%tmp)
    ax.set_xticks(xt)
    ax.set_xticklabels(xtl)
    ax.set_xlabel('month (day of year)', fontsize=14)

    ax.set_yticks([])
    
    plt.title('')

    plt.tight_layout()

    if sort_by_observability:
        plt.savefig('output/observability_'+prefix+'_byfrac_'+str(max_sunangle)+'dgr.pdf')
    else:
        plt.savefig('output/observability_'+prefix+'_byname_'+str(max_sunangle)+'dgr.pdf')